# HW2: Master info

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
import sys
from bs4 import BeautifulSoup as BSoup
import requests
formLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"
showLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247"
r = requests.get(formLink)
soup = BSoup(r.text, "lxml")

## Getting the data from ISAcademia

In [2]:
def showLinkGen(htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV):
  return "{}&{}={}&{}={}&{}={}&{}={}".format(showLink, htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV)

In [3]:
selectFields = soup.find_all("select")
infoField=selectFields[0].find("option", text="Informatique")["value"]
infoFieldParam=selectFields[0]["name"]
semParam=selectFields[2]["name"]
msSem1_text="Master semestre 1"
msSemPF_text="Projet Master automne"
msSemPS_text="Projet Master printemps"
msSem1=selectFields[2].find("option", text=msSem1_text)["value"]
msSemPF=selectFields[2].find("option", text=msSemPF_text)["value"]
msSemPS=selectFields[2].find("option", text=msSemPS_text)["value"]
allYears = [y["value"] for y in selectFields[1].find_all("option")[1:]]
yearParam = selectFields[1]["name"]
htmlradiobutton=soup.find("input", type="radio")
def get_html(sem, year):
    link = showLinkGen(htmlradiobutton["name"], htmlradiobutton["value"], infoFieldParam, infoField, yearParam, 
                 year, semParam, sem)
    return requests.get(link)

In [33]:
def get_ms_dataframe(request_link):
  soup2 = BSoup(request_link.text, "lxml")
  elems=soup2.find_all("tr")[2:]
  titleinfo = soup2.find("font")
  if(titleinfo is None):
      return pd.DataFrame([])
  else:
      titleinfo = titleinfo.text.split(', ')
      semester=titleinfo[2]
      semester=1 if semester==msSem1_text else (2 if semester==msSemPF_text else 3)
      year=titleinfo[1]
      all_data=[]
      for elem in elems:
        items=elem.find_all("td")
        gender = "M" if (items[0].text == "Monsieur") else "F"
        sciper = int(items[10].text)
        spec = items[4].text
        minor = items[6].text != ""
        all_data.append({"Scipper": sciper, "Sex": gender, "Year": year, "Semester": semester,
                        "Minor": minor, "Specialization": spec})
      return pd.DataFrame(all_data)

In [34]:
def get_ms_alldata():
  all_data = []
  for sem in [msSem1, msSemPF, msSemPS]:
    for year in allYears:
        all_data.append(get_ms_dataframe(get_html(sem, year)))
  return pd.concat(all_data)
ms_data = get_ms_alldata()
ms_data

,Minor,Scipper,Semester,Sex,Specialization,Year
0,False,271331,1,M,,2016-2017
1,False,273101,1,M,,2016-2017
2,False,267543,1,M,,2016-2017
3,False,202293,1,M,,2016-2017
4,False,228219,1,M,Foundations of Software,2016-2017
5,False,212464,1,M,,2016-2017
6,False,226371,1,M,Software Systems,2016-2017
7,False,273107,1,M,Foundations of Software,2016-2017
8,False,266983,1,M,,2016-2017
9,False,273118,1,M,,2016-2017


## Processing the data

In [35]:
ms_data.reset_index(None,drop=True,inplace=True)
ms_data.dtypes

Minor               bool
Scipper            int64
Semester           int64
Sex               object
Specialization    object
Year              object
dtype: object

In [36]:
master_semester1 = ms_data[ms_data.Semester == 1]
year = master_semester1['Year'].str.split('-',expand=True)
master_semester1.Year = year[0].astype(int)
master_semester1

/Users/asmaetounsi/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Minor,Scipper,Semester,Sex,Specialization,Year
0,False,271331,1,M,,2016
1,False,273101,1,M,,2016
2,False,267543,1,M,,2016
3,False,202293,1,M,,2016
4,False,228219,1,M,Foundations of Software,2016
5,False,212464,1,M,,2016
6,False,226371,1,M,Software Systems,2016
7,False,273107,1,M,Foundations of Software,2016
8,False,266983,1,M,,2016
9,False,273118,1,M,,2016


In [37]:
master_project = ms_data[ms_data.Semester != 1]
year = master_project['Year'].str.split('-',expand=True)
master_project.Year = np.where(master_project['Semester']==2, year[0].astype(int), year[1].astype(int))
master_project

/Users/asmaetounsi/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Minor,Scipper,Semester,Sex,Specialization,Year
948,False,166075,2,M,,2016
949,False,243163,2,M,,2016
950,False,194182,2,M,Foundations of Software,2016
951,True,213664,2,M,,2016
952,False,214637,2,M,,2016
953,False,243164,2,M,,2016
954,False,203947,2,M,,2016
955,True,203945,2,M,,2016
956,True,244136,2,M,,2016
957,False,229467,2,M,,2016


In [38]:
master_semester1.Scipper.is_unique

False

In [39]:
master_semester1 = master_semester1.sort_values("Year")
master_semester1 = master_semester1.drop_duplicates("Scipper", keep='first')
master_semester1.rename(columns = {'Year':'FirstYear'}, inplace=True)
master_semester1.drop('Specialization',axis=1, inplace=True)
master_semester1.drop('Minor',axis=1, inplace=True)
master_semester1.drop('Semester', axis=1, inplace=True)
master_semester1

,Scipper,Sex,FirstYear
947,166264,M,2007
895,160718,M,2007
896,138088,M,2007
897,154643,M,2007
898,166772,M,2007
899,167168,M,2007
900,166805,M,2007
894,153746,M,2007
901,154080,M,2007
903,180161,M,2007


In [40]:
master_project.Scipper.is_unique

False

In [41]:
master_project = master_project.sort_values("Year")
master_project = master_project.drop_duplicates("Scipper", keep='last')
master_project.rename(columns = {'Year':'LastYear'}, inplace=True)
master_project.Scipper.is_unique

True

In [42]:
master_semester1.head()

,Scipper,Sex,FirstYear
947,166264,M,2007
895,160718,M,2007
896,138088,M,2007
897,154643,M,2007
898,166772,M,2007


In [43]:
master_project.head()

,Minor,Scipper,Semester,Sex,Specialization,LastYear
1075,False,173527,3,M,,2008
1029,False,180027,2,F,,2008
1031,False,166805,2,M,,2008
1032,False,172264,2,M,,2008
1074,False,160150,3,M,Biocomputing,2008


In [44]:
ms = pd.merge(master_semester1,master_project, how='inner')
ms

,Scipper,Sex,FirstYear,Minor,Semester,Specialization,LastYear
0,166805,M,2007,False,2,,2008
1,172264,M,2007,False,2,,2008
2,165819,M,2007,False,3,Foundations of Software,2009
3,160225,M,2007,False,3,,2009
4,180027,F,2007,False,2,,2008
5,154573,F,2007,False,3,Biocomputing,2010
6,172687,M,2007,False,3,,2009
7,180072,M,2007,False,3,Internet computing,2009
8,166491,M,2007,False,3,,2011
9,177476,M,2007,False,3,,2009


In [45]:
ms['Staytime'] = (ms.LastYear - ms.FirstYear)*12 + (ms.Semester-1)*6
ms

,Scipper,Sex,FirstYear,Minor,Semester,Specialization,LastYear,Staytime
0,166805,M,2007,False,2,,2008,18
1,172264,M,2007,False,2,,2008,18
2,165819,M,2007,False,3,Foundations of Software,2009,36
3,160225,M,2007,False,3,,2009,36
4,180027,F,2007,False,2,,2008,18
5,154573,F,2007,False,3,Biocomputing,2010,48
6,172687,M,2007,False,3,,2009,36
7,180072,M,2007,False,3,Internet computing,2009,36
8,166491,M,2007,False,3,,2011,60
9,177476,M,2007,False,3,,2009,36


In [46]:
ms['Staytime'].mean()

28.789473684210527

In [47]:
ms_sex = ms.groupby('Sex')
ms_sex['Staytime'].mean()

Sex
F    27.692308
M    28.930693
Name: Staytime, dtype: float64

In [48]:
ms_spec = ms.groupby('Specialization')
ms_spec['Staytime'].mean()

Specialization
                                  27.204545
Biocomputing                      48.000000
Computer Engineering - SP         30.000000
Foundations of Software           36.000000
Information Security - SP         30.000000
Internet computing                34.200000
Service science                   30.000000
Signals, Images and Interfaces    36.000000
Software Systems                  30.000000
Name: Staytime, dtype: float64

In [49]:
ms_minor = ms.groupby('Minor')
ms_minor['Staytime'].mean()

Minor
False    28.666667
True     31.000000
Name: Staytime, dtype: float64

In [50]:
ms_minor_spec = ms.groupby(['Minor','Specialization'])
ms_minor_spec['Staytime'].mean()

Minor  Specialization                
False                                    26.926829
       Biocomputing                      48.000000
       Computer Engineering - SP         30.000000
       Foundations of Software           36.000000
       Information Security - SP         30.000000
       Internet computing                34.200000
       Service science                   30.000000
       Signals, Images and Interfaces    36.000000
       Software Systems                  30.000000
True                                     31.000000
Name: Staytime, dtype: float64

In [51]:
ms_minor_spec_sex = ms.groupby(['Sex','Minor','Specialization'])
ms_minor_spec_sex['Staytime'].mean()

Sex  Minor  Specialization                
F    False                                    24.60
            Biocomputing                      48.00
            Computer Engineering - SP         30.00
     True                                     36.00
M    False                                    27.25
            Computer Engineering - SP         30.00
            Foundations of Software           36.00
            Information Security - SP         30.00
            Internet computing                34.20
            Service science                   30.00
            Signals, Images and Interfaces    36.00
            Software Systems                  30.00
     True                                     30.00
Name: Staytime, dtype: float64

### statistical tests
* Specialization

In [73]:
import scipy.stats as stats

For each specialization, we apply a 1-sample T-Test, to see whether the results related to the staying-time average are significant or not.

In [75]:
for spec in ms.Specialization.unique():
    print(spec)
    print(stats.ttest_1samp(ms[ms.Specialization == spec].Staytime,ms.Staytime.mean()))


Ttest_1sampResult(statistic=-1.7137429312162629, pvalue=0.090136998432846185)
Foundations of Software
Ttest_1sampResult(statistic=3.894129279306477, pvalue=0.0080382847657293945)
Biocomputing
Ttest_1sampResult(statistic=nan, pvalue=nan)
Internet computing
Ttest_1sampResult(statistic=3.0058479532163749, pvalue=0.01481522720360739)
Signals, Images and Interfaces
Ttest_1sampResult(statistic=nan, pvalue=nan)
Service science
Ttest_1sampResult(statistic=nan, pvalue=nan)
Computer Engineering - SP
Ttest_1sampResult(statistic=inf, pvalue=0.0)
Information Security - SP
Ttest_1sampResult(statistic=inf, pvalue=0.0)
Software Systems
Ttest_1sampResult(statistic=inf, pvalue=0.0)


/Users/asmaetounsi/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


From the results obtained above, we notice that we do not have enough data to assume whether most of the staying-time averages according to specialization make a sens or not.
Therefore, some specialization categories' results make more sens then the others, according to this order:
   1. Students with no specialization.
   2. Students specialized in 'Internet computing'.
   3. Students specialized in 'Foundations of Software'.
   4. 'Computer Engineering - SP', 'Information Security - SP', 'Software Systems'.
   5. Others : 'Biocomputing', 'Signals, Images and Interfaces', 'Service science'